Import necessary libraries.

In [3]:
import glob
import os
import pandas as pd
import numpy as np
import datetime

First, merge/concatenate all datasets into one. Here, each daily dataset is not necessarily ordered by date, because of how they are stored in the "downloads" folder when initially downloaded with the dataset automation bot script. We also save the initial df into a csv file a contcatenated "raw" DataFrame (df).

Here, the current city name is just used as a placeholder. By specifying the correct file path, the datasets of the desired city can be used.

In [4]:
# Merge/Concatenate all datasets into one, specifying the correct location of all csv files for that city
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(r'C:\Users\I539797\Documents\Thesis\Datasets\Santiago', "*.csv"))))

In [5]:
# Save concatenated raw datasets into one csv file
concat_df = df.to_csv('concat_santiago.csv')

This is what the raw df looks like. We already get a hint of the necessity of having a multi-index structure: there are multiple Destination Movement IDs for one single Origin Movement ID and Date (which for now is called "Date Range").

This is because the travel times data is extracted from the center-most point/zone of each city to every other point/zone available at a given date.

The number of available points/zones varies fom date to date, therefore, there is a different amountof rows for each Origin Movement ID or date.

Later we will turn it into a multi-index df, indexed by date.

In [6]:
df.head(10)

,Origin Movement ID,Origin Display Name,Destination Movement ID,Destination Display Name,Date Range,Mean Travel Time (Seconds),Range - Lower Bound Travel Time (Seconds),Range - Upper Bound Travel Time (Seconds)
0,164,Movement Zone 164,24,Movement Zone 24,"1/3/2016 - 1/3/2016, Every day, Daily Average",437,318,599
1,164,Movement Zone 164,37,Movement Zone 37,"1/3/2016 - 1/3/2016, Every day, Daily Average",149,95,233
2,164,Movement Zone 164,122,Movement Zone 122,"1/3/2016 - 1/3/2016, Every day, Daily Average",203,145,283
3,164,Movement Zone 164,132,Movement Zone 132,"1/3/2016 - 1/3/2016, Every day, Daily Average",641,493,833
4,164,Movement Zone 164,133,Movement Zone 133,"1/3/2016 - 1/3/2016, Every day, Daily Average",298,179,493
5,164,Movement Zone 164,153,Movement Zone 153,"1/3/2016 - 1/3/2016, Every day, Daily Average",321,222,463
6,164,Movement Zone 164,157,Movement Zone 157,"1/3/2016 - 1/3/2016, Every day, Daily Average",116,35,376
7,164,Movement Zone 164,165,Movement Zone 165,"1/3/2016 - 1/3/2016, Every day, Daily Average",305,241,384
8,164,Movement Zone 164,182,Movement Zone 182,"1/3/2016 - 1/3/2016, Every day, Daily Average",160,50,504
9,164,Movement Zone 164,195,Movement Zone 195,"1/3/2016 - 1/3/2016, Every day, Daily Average",216,138,336


Perform the necessary operations to format the df, simplyfing columns names, dropping unnecessary columns, creating potentially relevant columns and most importantly, indexing and sorting the df by Date.

In [7]:
# Create a numeric index, because the existing index will be replaced by the Date Range later
df['Numeric Index'] = range(len(df))

# Reorder columns
df = df[['Numeric Index',
         'Origin Movement ID',
         'Origin Display Name',
         'Destination Movement ID',
         'Destination Display Name',
         'Date Range',
         'Mean Travel Time (Seconds)',
         'Range - Lower Bound Travel Time (Seconds)',
         'Range - Upper Bound Travel Time (Seconds)',
        ]]


# Reformat date and sort values by date
df['Date Range'] = pd.to_datetime(df['Date Range'].str.split().str[0], format='%m/%d/%Y')
df = df.sort_values('Date Range')


# Renaming the 'Date Range' column to 'Date'
df = df.rename(columns={"Date Range": "Date"})
# Removing sapces and odd character from column names
df = df.rename(columns={"Mean Travel Time (Seconds)": "MeanTravelTimeSeconds"})
df = df.rename(columns={"Numeric Index": "NumericIndex"})
df = df.rename(columns={"Origin Movement ID": "OriginMovementID"})
df = df.rename(columns={"Destination Movement ID": "DestinationMovementID"})
df = df.rename(columns={"Range - Lower Bound Travel Time (Seconds)": "RangeLowerBoundTravelTimeSeconds"})
df = df.rename(columns={"Range - Upper Bound Travel Time (Seconds)": "RangeUpperBoundTravelTimeSeconds"})


# Deleting columns that are not helpful for the analysis
df.drop('Origin Display Name', axis=1, inplace=True)
df.drop('Destination Display Name', axis=1, inplace=True)


# Set Date as the index
df.set_index(df['Date'], inplace=True)


# Create a 'DayofWeek' column
df['DayOfWeek'] = df['Date'].dt.day_name()
df.head(548)


# Drop the "Date" column since now, Date is the index
df.drop(['Date'], axis=1, inplace=True)

# Visualize head of final formatted df
df.head()

,NumericIndex,OriginMovementID,DestinationMovementID,MeanTravelTimeSeconds,RangeLowerBoundTravelTimeSeconds,RangeUpperBoundTravelTimeSeconds,DayOfWeek
Date,,,,,,,
2016-01-02,439079,164,548,537,496,580,Saturday
2016-01-02,439058,164,221,242,186,313,Saturday
2016-01-02,439057,164,219,788,658,942,Saturday
2016-01-02,439056,164,207,1064,531,2129,Saturday
2016-01-02,439055,164,195,255,166,391,Saturday


Lastly, save the formatted df into a new csv which will be used for analysis.

In [8]:
# Save formatted df into a csv file
formatted_df = df.to_csv('formatted_santiago.csv')